In [2]:
import pandas as pd
import plotly.graph_objects as go

In [3]:
scal_test_res = pd.read_pickle("scal_test_res.pkl")
all_trades = pd.read_pickle("all_trades.pkl")

In [4]:
all_trades.shape #NOTE: here we're still using the all trades calculate for moving averages


(15862, 11)

In [5]:
'''
The difference in the number of trades between the *moving average* strategy 
and the *RSI scalping* strategy is due to the nature of their *signals*.
*Moving average crossovers* occur frequently as prices oscillate around the 
averages, generating more trades in both *trending* and *ranging* markets. In 
contrast, the *RSI strategy* generates fewer signals because it only triggers
trades when the market reaches *overbought* or *oversold conditions* 
(e.g., RSI below 30 or above 70), which happen less often. As a result, the 
*moving average* strategy tends to produce more trades compared to the more selective
*RSI scalping* strategy.
'''
scal_test_res.shape

(45, 10)

In [6]:
'''
The *moving average* strategy often results in more negative total
gains compared to the *RSI scalping* strategy due to the frequent
generation of *false signals* in choppy or sideways markets.
When prices oscillate around the moving averages without a 
clear trend, the strategy can lead to many *whipsaw trades*
—where buy and sell signals are triggered close 
together—resulting in small, frequent losses that accumulate.
Additionally, in volatile markets, *moving averages* may 
not adapt quickly enough to price changes, causing late 
entries and exits that miss profitable opportunities or 
exacerbate losses. This contrasts with the *RSI scalping* 
strategy, which tends to be more selective and triggers 
trades only during extreme market conditions, potentially 
leading to fewer but more profitable trades.
'''
scal_test_res.head()

,pair,num_trades,total_gain,mean_gain,min_gain,max_gain,mean_duration,max_duration,min_duration,rsi_period
0,GBP_CAD,986,380.2,0.385598,-76.4,132.1,5.736308,131.0,1.0,14
1,GBP_CAD,631,1341.0,2.125198,-106.2,156.6,8.871632,169.0,1.0,21
2,GBP_CAD,387,390.4,1.008786,-111.1,179.1,13.956072,358.0,1.0,28
3,GBP_JPY,1220,-2440.5,-2.000410,-152.9,383.4,4.636066,160.0,1.0,14
4,GBP_JPY,897,-603.7,-0.673021,-207.5,359.6,6.243032,245.0,1.0,21


In [7]:
scal_test_res.columns

Index(['pair', 'num_trades', 'total_gain', 'mean_gain', 'min_gain', 'max_gain',
       'mean_duration', 'max_duration', 'min_duration', 'rsi_period'],
      dtype='object')

In [8]:
scal_test_res = scal_test_res[['pair', 'num_trades', 'total_gain', 'mean_duration', 'min_duration', 'max_duration', 'rsi_period']]

In [9]:
'''
Original Test Explanation:
The original line:
ma_test_res["CROSS"] = "MA_"+ma_test_res.mashort.map(str) +"_"+ma_test_res.malong.map(str) 
creates a column "CROSS" that concatenates the moving average 
periods (mashort and malong) into a string format 
"MA_mashort_malong". This is used to group and analyze results 
based on different combinations of moving average periods.

Why It's Not Fitting for RSI Scalping:
For the RSI scalping strategy, using the same "CROSS" column to
group results based on RSI parameters wouldn't be appropriate.
RSI
'''

scal_test_res["RSI_PERIOD"] = "RSI_" + scal_test_res.rsi_period.map(str)


In [10]:
# Group by RSI_PERIOD (instead of CROSS for moving averages) and relevant parameters
df_all_gains = scal_test_res.groupby(by=["RSI_PERIOD", "rsi_period"], as_index=False).sum()

# Select relevant columns for analysis
df_all_gains = df_all_gains[['RSI_PERIOD', 'num_trades', 'total_gain', 'rsi_period']]


In [11]:
df_all_gains.sort_values(by='total_gain', ascending=False, inplace=True)

In [12]:
df_all_gains.head()

,RSI_PERIOD,num_trades,total_gain,rsi_period
2,RSI_28,6453,5078.2,28
1,RSI_21,9860,2958.3,21
0,RSI_14,15421,930.4,14


In [13]:
scal_rsi_28 = scal_test_res[scal_test_res.RSI_PERIOD =='RSI_28'].copy()

In [14]:
total_p = len(scal_rsi_28.pair.unique())

In [15]:
scal_rsi_28[scal_rsi_28.total_gain>0].shape[0]

11

In [16]:
scal_rsi_28.head(10)

,pair,num_trades,total_gain,mean_duration,min_duration,max_duration,rsi_period,RSI_PERIOD
2,GBP_CAD,387,390.4,13.956072,1.0,358.0,28,RSI_28
5,GBP_JPY,658,-109.8,8.261398,1.0,281.0,28,RSI_28
8,GBP_NZD,337,1232.5,16.498516,1.0,336.0,28,RSI_28
11,GBP_CHF,469,-237.7,11.588486,1.0,477.0,28,RSI_28
14,EUR_GBP,306,93.0,17.980392,1.0,381.0,28,RSI_28
17,EUR_CAD,334,633.5,16.742515,1.0,503.0,28,RSI_28
20,EUR_JPY,588,499.2,9.175170,1.0,306.0,28,RSI_28
23,EUR_NZD,298,1344.9,18.627517,1.0,378.0,28,RSI_28
26,EUR_CHF,509,-311.4,10.996071,1.0,477.0,28,RSI_28
29,CAD_JPY,498,174.8,10.763052,1.0,299.0,28,RSI_28


In [17]:
for rsi_period in df_all_gains.RSI_PERIOD.unique():
    df_temp = scal_test_res[scal_test_res.RSI_PERIOD == rsi_period]
    #print(rsi_period)
    #print(df_temp.head())    
    total_p = df_temp.shape[0]
    n_good = df_temp[df_temp.total_gain>0].shape[0]
    print(f"{rsi_period:12} {n_good: 4} {(n_good/total_p)*100:4.0f}%")

RSI_28         11   73%
RSI_21          8   53%
RSI_14          9   60%


In [18]:
rsi_periods = df_all_gains.RSI_PERIOD.unique()[:3]
rsi_periods 

array(['RSI_28', 'RSI_21', 'RSI_14'], dtype=object)

In [19]:
df_good = scal_test_res[(scal_test_res.RSI_PERIOD.isin(rsi_periods))&(scal_test_res.total_gain>0)].copy()


In [20]:
df_good.head()

,pair,num_trades,total_gain,mean_duration,min_duration,max_duration,rsi_period,RSI_PERIOD
0,GBP_CAD,986,380.2,5.736308,1.0,131.0,14,RSI_14
1,GBP_CAD,631,1341.0,8.871632,1.0,169.0,21,RSI_21
2,GBP_CAD,387,390.4,13.956072,1.0,358.0,28,RSI_28
6,GBP_NZD,978,630.2,5.732106,1.0,119.0,14,RSI_14
7,GBP_NZD,585,1927.9,9.444444,1.0,169.0,21,RSI_21
